In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import sys 
import dotenv
sys.path.append("../../")

dotenv.load_dotenv("../.env")

True

In [8]:
USERNAME = "mytestuser1"
PASSWORD = "mytestpassword"
EMAIL = "mytestuser1@gmail.com"
FULL_NAME = "My Test User"

In [9]:
# first of all we sign up to the atria hub to manage our datasets and models
from atria_hub.hub import AtriaHub
hub = AtriaHub()
try:
    user = hub.auth.sign_up(email=EMAIL, password=PASSWORD, full_name=FULL_NAME, username=USERNAME)
except Exception:
    pass 

In [10]:
# after a successful sign up, we can initialize the hub with our login credentials
user = hub.auth.sign_in(email=EMAIL, password=PASSWORD)
hub.initialize()


Sign-in successful for mytestuser1@gmail.com


In [11]:
# now we first load the cifar10 dataset from the atria datasets predefiend registry
from atria_datasets import AtriaImageDataset
dataset = AtriaImageDataset.load_from_registry("cifar10")
# now lets upload the dataset to the atria hub
dataset.upload_to_hub('test-cifar10-5', branch='test')


[2025-07-19 17:13:51][atria_datasets.core.dataset.atria_dataset][INFO] Loading dataset cifar10 from registry.
[2025-07-19 17:13:51][atria_datasets.core.dataset.atria_dataset][INFO] Loading cached split train from /mnt/hephaistos/.atria/datasets/cifar10/main-01eefb81/delta/train
[2025-07-19 17:13:52][atria_datasets.core.dataset.atria_dataset][INFO] Loading cached split test from /mnt/hephaistos/.atria/datasets/cifar10/main-01eefb81/delta/test
[2025-07-19 17:13:52][atria_datasets.core.dataset.atria_dataset][WARNING] Split validation not found. Skipping.
[2025-07-19 17:13:52][atria_datasets.core.dataset.atria_dataset][INFO] Uploading dataset Cifar10 to hub with name test-cifar10-5 and config test.
[2025-07-19 17:13:52][atria_datasets.core.dataset.atria_dataset][ERROR] Failed to upload dataset to hub: This dataset already contains uploaded files in branch 'test'. Please use a different branch name or delete the existing files.


RuntimeError: This dataset already contains uploaded files in branch 'test'. Please use a different branch name or delete the existing files.

In [ ]:
# now lets upload the dataset to the atria hub
dataset.upload_to_hub('test-cifar10-4', branch='test')

[2025-07-19 17:06:34][atria_datasets.core.dataset.atria_dataset][INFO] Uploading dataset Cifar10 to hub with name test-cifar10-4 and config test.
[2025-07-19 17:06:35][atria_datasets.core.dataset.atria_dataset][ERROR] Failed to upload dataset to hub: This dataset already contains uploaded files in branch 'test'. Please use a different branch name or delete the existing files.


RuntimeError: This dataset already contains uploaded files in branch 'test'. Please use a different branch name or delete the existing files.

In [ ]:
# now lets upload the dataset to the atria hub
from atria_datasets.core.dataset.atria_dataset import AtriaImageDataset
dataset = AtriaImageDataset.load_from_hub(
    name=f"{hub.auth.username}/test-cifar10-4/test", streaming=True
)
print(dataset.metadata)
x = dataset.train[0]
print(x)

[2025-07-19 17:06:39][atria_datasets.core.dataset.atria_hub_dataset][INFO] Streaming dataset split train from lakefs://mytestuser1-26c51e41f3db44e1/test/delta/train/
[2025-07-19 17:06:39][atria_datasets.core.dataset.atria_hub_dataset][INFO] Streaming dataset split test from lakefs://mytestuser1-26c51e41f3db44e1/test/delta/test/
[2025-07-19 17:06:39][atria_datasets.core.dataset.atria_dataset][WARNING] Split validation not found. Skipping.


homepage=None description='CIFAR-10 dataset' license=None citation=None dataset_labels=DatasetLabels(
    classification=[
        'airplane',
        'automobile',
        'bird',
        'cat',
        'deer',
        'dog',
        'frog',
        'horse',
        'ship',
        'truck'
    ],
    ser=None,
    layout=None
)
ImageInstance(
    index=0,
    sample_id='09447729-0f39-4fb3-855f-9aedf183d106',
    image=Image(
        content=<PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32 at 0x71FBDD968050>,
        source_width=32,
        source_height=32,
        width=32,
        height=32,
        channels=3
    ),
    gt=GroundTruth(
        classification=ClassificationGT(label=Label(value=6, name='frog'))
    )
)


In [15]:
# now we build a resnet50 model pipeline for image classification from the atria models registry
from atria_models.pipelines.atria_model_pipeline import AtriaModelPipeline
model_pipeline = AtriaModelPipeline.load_from_registry(
    pipeline_name="image_classification",
    model_name="resnet50",
    dataset_metadata=dataset.metadata,
)



/mnt/hephaistos/projects/atria_agent/atria/.venv/lib/python3.11/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
[2025-07-19 17:14:09][atria_models.core.timm_model][INFO] Setting up model TimmModel(model_name='resnet50', num_classes=10)


In [16]:
# now we upload the model pipeline to the atria hub
model_pipeline.upload_to_hub(
    name="resnet50",
    branch="main",
)

[2025-07-19 17:14:11][atria_models.pipelines.atria_model_pipeline][INFO] Uploading model ImageClassificationPipeline to hub with name resnet50 and config main.


In [19]:
# now we load the model pipeline from the hub
name = f"{hub.auth.username}/resnet50/main"
print(name)
model_pipeline = AtriaModelPipeline.load_from_hub(
    name=f"{hub.auth.username}/resnet50/main",
)
print(model_pipeline)

[2025-07-19 17:14:51][atria_models.pipelines.atria_model_pipeline][INFO] Loading model mytestuser1/resnet50/main from hub.


mytestuser1/resnet50/main


[2025-07-19 17:14:52][atria_models.core.timm_model][INFO] Setting up model TimmModel(model_name='resnet50', num_classes=10)


ImageClassificationPipeline:
Layer (type:depth-idx)                             Param #
ModuleDict                                         --
├─TimmModel: 1-1                                   --
│    └─ResNet: 2-1                                 --
│    │    └─Conv2d: 3-1                            9,408
│    │    └─BatchNorm2d: 3-2                       128
│    │    └─ReLU: 3-3                              --
│    │    └─MaxPool2d: 3-4                         --
│    │    └─Sequential: 3-5                        215,808
│    │    └─Sequential: 3-6                        1,219,584
│    │    └─Sequential: 3-7                        7,098,368
│    │    └─Sequential: 3-8                        14,964,736
│    │    └─SelectAdaptivePool2d: 3-9              --
│    │    └─Linear: 3-10                           20,490
├─CrossEntropyLoss: 1-2                            --
├─CrossEntropyLoss: 1-3                            --
Total params: 23,528,522
Trainable params: 23,528,522
Non-trainable

In [20]:
from atria_core.types.factory import ImageInstanceFactory
from atria_core.types import GroundTruth
model_pipeline.enable_runtime_transforms()
image = ImageInstanceFactory.build()
image.gt = GroundTruth(classification = image.gt.classification)
model_pipeline.training_step([image])

[9, 2, 2, 0, 9, 1, 6, 2, 2, 3]
[7, 7, 2, 7, 2, 0, 8, 0, 7, 5]


ClassificationModelOutput(loss=tensor(2.3897, grad_fn=<NllLossBackward0>), logits=tensor([[ 0.0768, -0.3727,  0.1541, -0.0527, -0.4946,  0.3047,  0.2774,  0.1382,
         -0.0338,  0.0631]], grad_fn=<AddmmBackward0>), prediction=None, label=Label(value=tensor([3]), name=['light']))

In [ ]:
# from omegaconf import OmegaConf
# print(json.dumps(OmegaConf.to_container(model_pipeline.config), indent=2, sort_keys=True))

In [ ]:
# from atria_models.pipelines.classification.image import ImageClassificationPipeline

# print(model_pipeline.config)
# model_pipeline.upload_to_hub(
#     "mymodel"
# )

# loaded = ImageClassificationPipeline.load_from_hub(
#     "mymodel", 
# )

In [ ]:
# loaded.state_dict()['dataset_metadata'] = model_pipeline.state_dict()['dataset_metadata']
# loaded.state_dict()['config'] = model_pipeline.state_dict()['config']
# print(loaded.state_dict()['config'])

In [ ]:
# from atria_models.pipelines.classification.image import ImageClassificationPipeline
# from omegaconf import OmegaConf
# from atria_models.registry import MODEL
# from atria_models.core.timm_model import TimmModel
# from rich.pretty import pretty_repr
# import json
# model = TimmModel(model_name="resnet50")
# pipeline = ImageClassificationPipeline(model=model, checkpoint_configs='test')
# print(json.dumps(pipeline._config, indent=2))